In [1]:
ns_kicks = [400.0, 450.0, 500.0, 550.0, 600.0, 650.0]
sigma_kicks = [0.1, 0.2, 0.3, 0.4, 0.5]

In [18]:
import math
import os

In [3]:
def getSbatch(n_systems, bh_kick, ns_kick = 400, sigma_kicks = [0.3], run_size=2500, job_name='bse_evolution', queue='development', timeout = '02:00:00'):  
    n_systems = int(n_systems)
    run_size = int(run_size)
    
    n_runs = int(n_systems/run_size)
    
    tasks = n_runs*len(sigma_kicks)
    nodes = max(1, math.ceil(tasks/56))
    
        
    preamble_text = f"""#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J {job_name}           # Job name
#SBATCH -o {job_name}.o%j	   # Name of stdout output file
#SBATCH -e {job_name}.e%j	   # Name of stderr error file
#SBATCH -p {queue}          # Queue (partition) name
#SBATCH -N {nodes}             # Total # of nodes
#SBATCH -n {tasks}             # Total # of mpi tasks
#SBATCH -t {timeout}       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

export I_MPI_DEBUG=4

# Launch MPI code...
"""
    for sigma_kick in sigma_kicks:
        preamble_text+= f'mkdir $WORK/supernova_remnant_bse/bh_{bh_kick}_ns_{ns_kick}_sigma_{sigma_kick}\n' 

    preamble_text += f"\nfor i in `seq 1 {n_runs}` \ndo \n"
    
    for sigma_kick in sigma_kicks:
        output_path = f'$WORK/supernova_remnant_bse/bh_{bh_kick}_ns_{ns_kick}_sigma_{sigma_kick}'
        specs = f" --number-of-systems {run_size} --output-path {output_path}/ --mode BSE --evolve-unbound-systems TRUE --kick-magnitude-distribution MULLERMANDEL --remnant-mass-prescription MULLERMANDEL --muller-mandel-kick-multiplier-BH {bh_kick} --muller-mandel-kick-multiplier-NS {ns_kick} --muller-mandel-sigma-kick {sigma_kick}"
        preamble_text += f"    ibrun -n 1 ~/supernova_remnant/COMPAS/src/COMPAS" + specs + f" --output-container run_$i & \n"
    
    preamble_text += "done \nwait"
    
    return preamble_text



In [6]:
n_systems = 1e6
bh_kick = 200.0
# ns_kicks = [500.0]
# sigma_kicks = [0.1, 0.2, 0.3, 0.4, 0.5]

for ns_kick in ns_kicks:
    job_name = f'bh_{bh_kick}_ns_{ns_kick}'
    sbatch_text = getSbatch(n_systems, bh_kick, ns_kick, sigma_kicks=sigma_kicks, run_size = 25000, job_name=job_name, queue='normal', timeout='4:00:00')
    file1 = open(f"../sbatch_bse_supernova/{job_name}", "w")  # write mode
    file1.write(sbatch_text)
    file1.close()
        
print(sbatch_text)

#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J bh_200.0_ns_650.0           # Job name
#SBATCH -o bh_200.0_ns_650.0.o%j	   # Name of stdout output file
#SBATCH -e bh_200.0_ns_650.0.e%j	   # Name of stderr error file
#SBATCH -p normal          # Queue (partition) name
#SBATCH -N 4             # Total # of nodes
#SBATCH -n 200             # Total # of mpi tasks
#SBATCH -t 4:00:00       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

export I_MPI_DEBUG=4

# Launch MPI code...
mkdir $WORK/supernova_remnant_bse/bh_200.0_ns_650.0_sigma_0.1
mkdir $WORK/supernova_remnant_bse/bh_200.0_ns_650.0_sigma_0.2
mkdir $WORK/super

In [22]:
for ns_kick in [400.0, 450.0, 550.0, 600.0, 650.0]:
# for ns_kick in [400.0]:
    print()
    for sigma in sigma_kicks:
        print(f"$WORK/h5copy.py -o $WORK/supernova_remnant_bse/bh_200_ns_{ns_kick:.0f}_sigma_{sigma:.1f}_combined.h5 input $WORK/supernova_remnant_bse/bh_200.0_ns_{ns_kick:.1f}_sigma_{sigma:.1f}/ -r ")
        
        
        
        


$WORK/h5copy.py -o $WORK/supernova_remnant_bse/bh_200_ns_400_sigma_0.1_combined.h5 input $WORK/supernova_remnant_bse/bh_200.0_ns_400.0_sigma_0.1/ -r 
$WORK/h5copy.py -o $WORK/supernova_remnant_bse/bh_200_ns_400_sigma_0.2_combined.h5 input $WORK/supernova_remnant_bse/bh_200.0_ns_400.0_sigma_0.2/ -r 
$WORK/h5copy.py -o $WORK/supernova_remnant_bse/bh_200_ns_400_sigma_0.3_combined.h5 input $WORK/supernova_remnant_bse/bh_200.0_ns_400.0_sigma_0.3/ -r 
$WORK/h5copy.py -o $WORK/supernova_remnant_bse/bh_200_ns_400_sigma_0.4_combined.h5 input $WORK/supernova_remnant_bse/bh_200.0_ns_400.0_sigma_0.4/ -r 
$WORK/h5copy.py -o $WORK/supernova_remnant_bse/bh_200_ns_400_sigma_0.5_combined.h5 input $WORK/supernova_remnant_bse/bh_200.0_ns_400.0_sigma_0.5/ -r 

$WORK/h5copy.py -o $WORK/supernova_remnant_bse/bh_200_ns_450_sigma_0.1_combined.h5 input $WORK/supernova_remnant_bse/bh_200.0_ns_450.0_sigma_0.1/ -r 
$WORK/h5copy.py -o $WORK/supernova_remnant_bse/bh_200_ns_450_sigma_0.2_combined.h5 input $WORK/sup

# Detection Rate comparison

In [17]:
def getSbatch_det(n_systems, kick_dist, bh_kick, ns_kick = 550, sigma_kick = 0.4, run_size=2500, job_name='det_rates', queue='development', timeout = '02:00:00'):  
    n_systems = int(n_systems)
    run_size = int(run_size)
    
    n_runs = int(n_systems/run_size)
    
    tasks = n_runs
    nodes = max(1, math.ceil(tasks/56))
    
    run_size = math.ceil(n_systems/(nodes*56))
    
        
    preamble_text = f"""#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J {job_name}           # Job name
#SBATCH -o {job_name}.o%j	   # Name of stdout output file
#SBATCH -e {job_name}.e%j	   # Name of stderr error file
#SBATCH -p {queue}          # Queue (partition) name
#SBATCH -N {nodes}             # Total # of nodes
#SBATCH -n {nodes*56}             # Total # of mpi tasks
#SBATCH -t {timeout}       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

export I_MPI_DEBUG=4

# Launch MPI code...
"""

    preamble_text+= f'mkdir $SCRATCH/supernova_remnant_det/{job_name}\n' 

    preamble_text += f"\nfor i in `seq 1 {nodes*56}` \ndo \n"
    
    if kick_dist=='mullermandel':
        output_path = f'$SCRATCH/supernova_remnant_det/{job_name}'
        specs = f" --number-of-systems {run_size} --output-path {output_path}/ --mode BSE --evolve-unbound-systems TRUE --metallicity-distribution LOGUNIFORM --kick-magnitude-distribution MULLERMANDEL --remnant-mass-prescription MULLERMANDEL --muller-mandel-kick-multiplier-BH {bh_kick} --muller-mandel-kick-multiplier-NS {ns_kick} --muller-mandel-sigma-kick {sigma_kick}"
        preamble_text += f"    ibrun -n 1 ~/supernova_remnant/COMPAS/src/COMPAS" + specs + f" --output-container run_$i & \n"
    
    if kick_dist=='hobbs':
        output_path = f'$SCRATCH/supernova_remnant_det/{job_name}'
        specs = f" --number-of-systems {run_size} --output-path {output_path}/ --mode BSE --evolve-unbound-systems TRUE --metallicity-distribution LOGUNIFORM"
        preamble_text += f"    ibrun -n 1 ~/supernova_remnant/COMPAS/src/COMPAS" + specs + f" --output-container run_$i & \n"
    
    preamble_text += "done \nwait \n"
    
#     preamble_text += f"$WORK/h5copy.py -o $WORK/supernova_remnant_det/{job_name}_combined.h5 $WORK/supernova_remnant_det/{job_name}/ -r "
    
    return preamble_text



In [23]:
n_systems = 2e7
bh_kick=200
ns_kick=500
sigma_kick=0.4

kick_dist='mullermandel'
job_name = f'{kick_dist}_ns_{ns_kick}_sigma_{sigma_kick}_detection_2e7_1'

sbatch_text = getSbatch_det(n_systems, kick_dist, bh_kick, ns_kick, sigma_kick, run_size = 100000, job_name=job_name, queue='normal', timeout='12:00:00')

file1 = open(f"{os.environ['SCRATCH']}/sbatch_det_rates/{job_name}", "w")  # write mode
file1.write(sbatch_text)
file1.close()

print(sbatch_text)

#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J mullermandel_ns_500_sigma_0.4_detection_2e7_1           # Job name
#SBATCH -o mullermandel_ns_500_sigma_0.4_detection_2e7_1.o%j	   # Name of stdout output file
#SBATCH -e mullermandel_ns_500_sigma_0.4_detection_2e7_1.e%j	   # Name of stderr error file
#SBATCH -p normal          # Queue (partition) name
#SBATCH -N 4             # Total # of nodes
#SBATCH -n 224             # Total # of mpi tasks
#SBATCH -t 12:00:00       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

export I_MPI_DEBUG=4

# Launch MPI code...
mkdir $SCRATCH/supernova_remnant_det/mullermandel_ns_500

In [22]:
n_systems = 2e7
kick_dist='hobbs'
job_name = f'{kick_dist}_detection_2e7_1'

sbatch_text = getSbatch_det(n_systems, kick_dist, bh_kick, ns_kick, sigma_kick, run_size = 100000, job_name=job_name, queue='normal', timeout='12:00:00')
file1 = open(f"{os.environ['SCRATCH']}/sbatch_det_rates/{job_name}", "w")  # write mode
file1.write(sbatch_text)
file1.close()
print(sbatch_text)

#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J hobbs_detection_2e7_1           # Job name
#SBATCH -o hobbs_detection_2e7_1.o%j	   # Name of stdout output file
#SBATCH -e hobbs_detection_2e7_1.e%j	   # Name of stderr error file
#SBATCH -p normal          # Queue (partition) name
#SBATCH -N 4             # Total # of nodes
#SBATCH -n 224             # Total # of mpi tasks
#SBATCH -t 12:00:00       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

export I_MPI_DEBUG=4

# Launch MPI code...
mkdir $SCRATCH/supernova_remnant_det/hobbs_detection_2e7_1

for i in `seq 1 224` 
do 
    ibrun -n 1 ~/supernova_remnant/COMPAS/

In [1]:
mullermandel_path = "mullermandel_ns_500_sigma_0.4_detection"
mullermadel_out = "mullermandel_combined.h5"

hobbs_path = "hobbs_detection"
hobbs_out = "hobbs_det_combined.h5"

print(f"$WORK/h5copy.py -o $SCRATCH/supernova_remnant_det/{mullermandel_path} input $SCRATCH/supernova_remnant_det/{mullermandel_path}/ -r ")
print()
print(f"$WORK/h5copy.py -o $SCRATCH/supernova_remnant_det/{hobbs_out} input $SCRATCH/supernova_remnant_det/{hobbs_path}/ -r ")


$WORK/h5copy.py -o $SCRATCH/supernova_remnant_det/mullermandel_200_ns_550_detection input $SCRATCH/supernova_remnant_det/mullermandel_200_ns_550_detection/ -r 

$WORK/h5copy.py -o $SCRATCH/supernova_remnant_det/hobbs_det_combined.h5 input $SCRATCH/supernova_remnant_det/hobbs_detection/ -r 
